# Explorlatory data analysis on the content of the datasets

In [1]:
import pandas as pd
import plotly.express as px


## ETF
All traded in EUR. All from Borsa di Milano, except for the Crypto ETFs.

I will exclude crypto from EDA because they lack some info and there are better way to get crypto prices.

Columns of `/dati/{ticker}.csv` are:
- Date
- OPEN_PRC
- HIGH_1
- LOW_1
- TRDPRC_1 (Close)
- TRNOVR_UNS (Volume)

In [2]:
df_info = (
    pd.read_excel("../ETF/elenco ETF.xlsx")
    [["RIC", "ISIN", "Name", "Lipper Classification Scheme", "Instrument Type", "Domicile"]] # interesting info
    .rename(columns={"RIC": "Ticker", "Name": "ETF Name", "Lipper Classification Scheme": "Category", "Instrument Type": "Type"})
    .dropna(subset=["ISIN"]) # this will exclude 4 crypto ETCs from xetra
)
df_info

,Ticker,ISIN,ETF Name,Category,Type,Domicile
0,MMLP.MI,IE00BKPTXQ89,Alerian Midstream Energy Dividend UCITS ETF Dis,Equity Theme - Infrastructure,Equity ETF,Ireland
1,FAMSFFA.MI,IE000NPK2VI6,AM MSCI EUR IG SRI SusExFossil CrpBd UCITS ETF,Bond EUR Corporates,Exchange-Traded Fund,Ireland
2,FAMSF3A.MI,IE000W6ZFH20,AM MSCI EUR IG SRI SusExFossil1-3yCrpBdUCITSETF,Bond EUR Corporates Short Term,Exchange-Traded Fund,Ireland
3,COMH.MI,LU1900069219,Amndi Blmbrg Eqlwght ComdtexAgrcl UCITSETF DHE...,Commodity Blended,Commodity ETF,Luxembourg
4,COMO.MI,LU1829218749,Amundi Blbrg Equlwght Commdt ex-Agricl UCITS E...,Commodity Blended,Commodity ETF,Luxembourg
...,...,...,...,...,...,...
1816,XDGU.MI,IE00BZ036H21,Xtrackers USD Corporate Bond UCITS ETF 1D,Bond USD Corporates,Bond ETF,Ireland
1817,XDGE.MI,IE00BZ036J45,Xtrackers USD Corporate Bond UCITS ETF 2D EUR Hgd,Bond USD Corporates,Bond ETF,Ireland
1818,XGBU.MI,IE0003W9O921,Xtrackers USD Corporate Green Bond UCITS ETF 2C,NaN,Bond ETF,Ireland
1819,XUHY.MI,IE00BDR5HM97,Xtrackers USD High Yld Corporate Bond UCITS ET...,Bond USD High Yield,Bond ETF,Ireland


In [3]:
df_eda = pd.DataFrame(index=df_info["Ticker"], columns=["Date Oldest", "Date Newest", "Years Covered", "Data per Year", "Avg Volume (M EUR)"])
for ticker in df_info["Ticker"]:
    try:
        df_hist = (
            pd.read_csv(f"../ETF/dati/{ticker}.csv", sep=";", parse_dates=["Date"])
            .dropna(subset=["TRDPRC_1"])
        )
        oldest = df_hist["Date"].min().date()
        newest = df_hist["Date"].max().date()
        years_covered = (newest - oldest).days / 365.25
        data_per_year = len(df_hist) / years_covered
        avg_volume = (df_hist["TRNOVR_UNS"]*df_hist["TRDPRC_1"]).mean() / 1e6
        df_eda.loc[ticker] = [oldest, newest, years_covered, data_per_year, avg_volume]
    except FileNotFoundError:
        print(f"{ticker}: no data")
    except Exception as e:
        print(f"{ticker}: error {e}")

MIVO.MI: no data
ISAD.MI: no data
FMQQ.MI: no data
WTCHIN.MI: no data
EGTD.MI: error Missing column provided to 'parse_dates': 'Date'
MLPI.MI: no data
FLOTH.MI: no data
LCUK.MI: no data
FLOT.MI: no data
OIL1L.MI: no data
SGIOIL2L.MI: no data
EUCR.MI: no data
EXCPA.MI: no data
SIL1L.MI: no data
GGE.MI: error Missing column provided to 'parse_dates': 'Date'
USESGA.MI: error Missing column provided to 'parse_dates': 'Date'
JPEUB.MI: error float division by zero


In [4]:
df_eda2 = df_info.join(df_eda, on="Ticker", how="left")
df_eda2.to_csv("01_eda_ETF.csv", index=False)    
df_eda2

,Ticker,ISIN,ETF Name,Category,Type,Domicile,Date Oldest,Date Newest,Years Covered,Data per Year,Avg Volume (M EUR)
0,MMLP.MI,IE00BKPTXQ89,Alerian Midstream Energy Dividend UCITS ETF Dis,Equity Theme - Infrastructure,Equity ETF,Ireland,2020-08-07,2023-11-10,3.258042,210.862815,0.492546
1,FAMSFFA.MI,IE000NPK2VI6,AM MSCI EUR IG SRI SusExFossil CrpBd UCITS ETF,Bond EUR Corporates,Exchange-Traded Fund,Ireland,2022-10-10,2023-11-10,1.084189,105.147727,2.160326
2,FAMSF3A.MI,IE000W6ZFH20,AM MSCI EUR IG SRI SusExFossil1-3yCrpBdUCITSETF,Bond EUR Corporates Short Term,Exchange-Traded Fund,Ireland,2022-10-10,2023-11-10,1.084189,117.138258,1.628128
3,COMH.MI,LU1900069219,Amndi Blmbrg Eqlwght ComdtexAgrcl UCITSETF DHE...,Commodity Blended,Commodity ETF,Luxembourg,2019-04-15,2023-09-28,4.454483,207.880455,5.451044
4,COMO.MI,LU1829218749,Amundi Blbrg Equlwght Commdt ex-Agricl UCITS E...,Commodity Blended,Commodity ETF,Luxembourg,2006-06-01,2023-11-10,17.442847,254.144287,15.223571
...,...,...,...,...,...,...,...,...,...,...,...
1816,XDGU.MI,IE00BZ036H21,Xtrackers USD Corporate Bond UCITS ETF 1D,Bond USD Corporates,Bond ETF,Ireland,2016-02-22,2023-11-10,7.715264,202.844659,1.093141
1817,XDGE.MI,IE00BZ036J45,Xtrackers USD Corporate Bond UCITS ETF 2D EUR Hgd,Bond USD Corporates,Bond ETF,Ireland,2016-09-01,2023-11-10,7.189596,201.819402,2.643452
1818,XGBU.MI,IE0003W9O921,Xtrackers USD Corporate Green Bond UCITS ETF 2C,NaN,Bond ETF,Ireland,2021-09-08,2023-09-01,1.979466,24.248963,0.591846
1819,XUHY.MI,IE00BDR5HM97,Xtrackers USD High Yld Corporate Bond UCITS ET...,Bond USD High Yield,Bond ETF,Ireland,2018-04-27,2023-11-10,5.538672,205.825507,0.880885


In [5]:
# Olders ETFs
fig = px.histogram(df_eda2, x="Date Oldest", nbins=50)
fig.show()
display(df_eda2.sort_values("Date Oldest").head(10))

,Ticker,ISIN,ETF Name,Category,Type,Domicile,Date Oldest,Date Newest,Years Covered,Data per Year,Avg Volume (M EUR)
773,EUN.MI,IE0008470928,iShares STOXX Europe 50 UCITS ETF EUR (Dist),Equity Europe,Equity ETF,Ireland,2002-09-30,2023-11-10,21.111567,253.841882,27.737277
654,EUE.MI,IE0008471009,iShares Core EURO STOXX 50 UCITS ETF EUR Dist,Equity EuroZone,Equity ETF,Ireland,2002-09-30,2023-09-28,20.99384,254.122164,231.401299
942,MSE.MI,FR0007054358,Lyxor EURO STOXX 50 (DR) UCITS ETF - Acc,Equity EuroZone,Equity ETF,France,2002-09-30,2023-11-10,21.111567,254.173454,167.177967
1018,USTE.MI,LU1829221024,Lyxor Nasdaq-100 UCITS ETF - Acc,Equity US,Equity ETF,Luxembourg,2002-11-22,2023-09-28,20.848734,254.020223,13.441018
920,DJE.MI,FR0007056841,Lyxor Dow Jones Industrial Average UCITS ETF Dist,Equity US,Equity ETF,France,2002-11-22,2023-11-10,20.966461,253.97705,48.002654
666,IUSA.MI,IE0031442068,iShares Core S&P 500 UCITS ETF USD (Dist),Equity US,Equity ETF,Ireland,2003-02-28,2023-09-28,20.580424,254.173573,72.575498
497,EQQQ.MI,IE0032077012,Invesco EQQQ NASDAQ-100 UCITS ETF Dist,Equity US,Equity ETF,Ireland,2003-06-26,2023-09-28,20.257358,254.228612,185.262009
919,MGT.MI,FR0007075494,Lyxor DJ Global Titans 50 UCITS ETF - Dist,Equity Global,Equity ETF,France,2003-10-22,2023-11-10,20.052019,254.089125,3.082989
958,ETFMIB.MI,FR0010010827,Lyxor FTSE MIB (DR) UCITS ETF - Dist,Equity Italy,Equity ETF,France,2003-11-14,2023-09-28,19.871321,254.135092,230.763268
626,IBCX.MI,IE0032523478,iShares € Corp Bond Large Cap UCITS ETF EUR Dist,Bond EUR Corporates,Bond ETF,Ireland,2004-04-08,2023-11-10,19.589322,254.322222,238.571944


In [6]:
# Amount of data per year
df = df_eda2.copy()
df["Data total"] = df["Data per Year"] * df["Years Covered"]
df["NAME"] = df["Ticker"] + " - " + df["ETF Name"]

fig = px.scatter(df, x="Date Oldest", y="Data total", hover_name="NAME", hover_data=["Years Covered", "Data per Year", "Avg Volume (M EUR)"])
fig.show()

In [7]:
# Most traded ETFs
fig = px.histogram(df_eda2, x="Avg Volume (M EUR)", nbins=50)
fig.show()
display(df_eda2.sort_values("Avg Volume (M EUR)", ascending=False).head(10))

,Ticker,ISIN,ETF Name,Category,Type,Domicile,Date Oldest,Date Newest,Years Covered,Data per Year,Avg Volume (M EUR)
1586,3NGS.MI,IE00B76BRD76,Wisdomtree Natural Gas 3X Daily Short,NaN,Exchange-Traded Commodity,Ireland,2013-11-04,2023-09-28,9.897331,254.108921,613327.535936
1585,3NGL.MI,IE00BLRPRG98,Wisdomtree Natural Gas 3X Daily Leverage,NaN,Exchange-Traded Commodity,Ireland,2013-11-04,2023-09-28,9.897331,254.108921,523462.949795
1115,GAS3L.MI,XS2425314189,SG ETC NATURAL GAS +3X DAILY LEV COLLAT ETC,NaN,Exchange-Traded Commodity,NaN,2014-10-20,2023-09-28,8.939083,254.276646,20909.109971
1609,VIXL.MI,IE00BLRPRH06,Wisdomtree S&P 500 Vix Short-Term Future,NaN,Exchange-Traded Note,NaN,2016-03-17,2023-09-28,7.531828,254.652672,18345.985426
1588,LNGA.MI,JE00BDD9Q956,Wisdomtree Natural Gas Leverage ETC,NaN,Exchange-Traded Commodity,NaN,2009-05-26,2023-11-10,14.45859,254.519977,8621.195489
1501,3BRS.MI,IE00BLRPRK35,Wisdomtree Brent Crude Oil 3X Daily Shor,NaN,Exchange-Traded Commodity,NaN,2015-10-19,2023-09-28,7.942505,254.579628,3429.963516
665,CSSPX.MI,IE00B5BMR087,iShares Core S&P 500 UCITS ETF USD (Acc),Equity US,Equity ETF,Ireland,2010-05-27,2023-09-28,13.338809,254.370536,1032.197367
1583,QQQS.MI,IE00BLRPRJ20,Wisdomtree Nasdaq 100 3X Daily Short,NaN,Exchange-Traded Note,NaN,2014-02-25,2023-09-28,9.587953,254.38171,684.265179
768,CSNDX.MI,IE00B53SZB19,iShares NASDAQ 100 UCITS ETF USD (Acc),Equity US,Equity ETF,Ireland,2010-03-15,2023-09-28,13.538672,254.308544,627.091584
572,SPXS.MI,IE00B3YCGJ38,Invesco S&P 500 UCITS ETF Acc,Equity US,Equity ETF,Ireland,2014-09-29,2023-09-28,8.996578,249.872794,607.183575
